In [49]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

# FUNCTION TO COMPUTE DFF
import os
from utility_classification import sum_pixels_in_registered_mask, fix_trials



Autosaving every 180 seconds


In [50]:
import string, re
from scipy.signal import butter, filtfilt, cheby1


def make_comparison_movies(data_mean_copy, 
                           data_mean_filtered_copy,
                           name,
                           start,
                           end,
                           fps):
    #
    import matplotlib.animation as animation
    import numpy as np

    fname_out = '/home/cat/'+name+'.mp4'

    dpi = 100

    fig = plt.figure()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)

    ax1.set_aspect('equal')
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)

    ax2.set_aspect('equal')
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)

    vmax1 = np.percentile(data_mean_copy.ravel(),99.5)
    vmin1 = -vmax1

    vmax2 = np.percentile(data_mean_filtered_copy.ravel(),99.5)
    vmin2 = -vmax2


    # 
    #name = "AQ2 (GCamp6f mouse)"

    # 
    im1 = ax1.imshow(data_mean_copy[0],
                   vmax=vmax1,
                   vmin=vmin1,
                   cmap='viridis')

    im2 = ax2.imshow(data_mean_filtered_copy[0],
                   vmax=vmax2,
                   vmin=vmin2,
                   cmap='viridis')

    #plt.suptitle,fontsize=20)
    ax1.set_title('Original')
    ax2.set_title('Filtered')


    def update_img(n):
        plt.suptitle(name + "\n"+str( round((start+n)/30.-10.,2)) + "sec")
        im1.set_data(data_mean_copy[n])
        im2.set_data(data_mean_filtered_copy[n])
        return im1, im2

    #legend(loc=0)
    ani = animation.FuncAnimation(fig,
                                  update_img,
                                  data_mean_copy.shape[0],
                                  interval=30)
    
    writer = animation.writers['ffmpeg'](fps=fps)

    ani.save(fname_out,writer=writer,dpi=dpi)
    
def sigmoid_function(x, a, b):

    return np.clip(a*(np.ma.log(x) - np.ma.log(1 - x))+b, 0, 1)      #Compute sigmoid and cut off values below 0 and above 1
    
    
def mangle(width, x, img_temp, maxval, power, val999):
    
    mu = 0 #Select approximate midline as centre of gaussian
    sig = width
    
    a = .005       #The steepness of the sigmoid function
    b = val999        #% of maxval to cutoff


    #Normalize img_temp for sigmoid to work properly
    #img_temp_norm = (img_temp-np.min(img_temp))/(np.max(img_temp) - np.min(img_temp))
    img_temp_norm = (img_temp-np.min(img_temp))/(np.max(img_temp) - np.min(img_temp))


    #Original root function
    #return -np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))) * (abs(pix_val/maxval)**(1./power))
    return -np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.))) * sigmoid_function(img_temp_norm, a, b)


# FUNCTION TO MASK
def motion_mask_parallel(img_temp, maxval, val999, width, power):
    '''Parallel computation of mask
    '''
   
    y_array = []
    for x in range(len(img_temp)):
        y_array.append(np.arange(0,len(img_temp), 1))
        
    y_array = np.vstack(y_array)
    motion_mask = img_temp*mangle(width, np.abs(64-y_array), img_temp, maxval, power, val999)
    
                   
    motion_mask = (motion_mask-np.min(motion_mask))/(np.max(motion_mask)-np.min(motion_mask))

    idx = np.where(motion_mask==0)
    motion_mask[idx]=1
    
    #motion_mask = motion_mask * img_temp
        
    return motion_mask


In [51]:
fname_04 = '/media/cat/4TBSSD/yuki/AQ2/tif_files/AQ2pm_Dec16_30Hz/AQ2pm_Dec16_30Hz_code_04_trial_ROItimeCourses_10sec_all_brain.npy'
       
data = np.load(fname_04)
print (data.shape)
# compute mean

data_mean = data.mean(0)
print (data_mean.shape)

(27, 601, 128, 128)
(601, 128, 128)


In [62]:
#
img_temp = data_mean[301]
print (np.max(img_temp), np.min(img_temp))
maxval = 10  # NOT BEING USED CURRENTLY
# val999 = .98
# width = 18
# power = 1.8

import yaml

with open("/media/cat/4TBSSD/yuki/IJ2/gcamp.txt", 'r') as f:
    valuesYaml = yaml.load(f, Loader=yaml.FullLoader)

val999 = valuesYaml['val999']
width = valuesYaml['width']
power = valuesYaml['power']


img_out = motion_mask_parallel(img_temp.copy(), maxval, val999, width, power)

# plot
fig=plt.figure()
plt.suptitle("t=0 frame")
ax=plt.subplot(1,2,1)
plt.imshow(img_temp)
ax=plt.subplot(1,2,2)
plt.imshow(img_out*img_temp)
plt.show()


0.05237212893187076 -0.01166658908224142


FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/4TBSSD/yuki/IJ2/gcamp2.txt'

In [59]:
from tqdm import trange

# maxval = 10  # NOT BEING USED CURRENTLY
# val999 = .90
# width = 18
# power = 1.8

import yaml

with open("/media/cat/4TBSSD/yuki/IJ2/gcamp.txt", 'r') as f:
    valuesYaml = yaml.load(f, Loader=yaml.FullLoader)

val999 = valuesYaml['val999']
width = valuesYaml['width']
power = valuesYaml['power']


data_mean_filtered = data_mean.copy()
temp = motion_mask_parallel(data_mean_filtered[300:315].mean(0), maxval, val999, width, power)
#temp = motion_mask_parallel(data_mean_filtered[150:160].mean(0), maxval, val999, width, power)

for k in trange(data_mean_filtered.shape[0]):
    #data_mean_filtered[k] = motion_mask_parallel(data_mean_filtered[k], maxval, val999, width, power)

    data_mean_filtered[k] *=temp


100%|██████████| 601/601 [00:00<00:00, 61552.92it/s]


In [60]:
# 

name = 'AQ2'

start = 200
end = start+200
fps = 5

data_mean_copy = data_mean.copy()[start:end]
data_mean_filtered_copy = data_mean_filtered.copy()[start:end]



make_comparison_movies(data_mean_copy, 
                       data_mean_filtered_copy,
                      name,
                      start,
                      end,
                      fps)


In [73]:
fname = '/media/cat/4TBSSD/yuki/AQ2/tif_files.npy'
data1 = np.load(fname)
print (data1)
# np.save(fname[:-4]+"_old.npy", data1)

# np.save(fname, data1[:44])


[b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2am_Dec9_30Hz/AQ2am_Dec9_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2am_Dec10_30Hz/AQ2am_Dec10_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2pm_Dec10_30Hz/AQ2pm_Dec10_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2am_Dec11_30Hz/AQ2am_Dec11_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2pm_Dec14_30Hz/AQ2pm_Dec14_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2am_Dec14_30Hz/AQ2am_Dec14_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2pm_Dec16_30Hz/AQ2pm_Dec16_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2am_Dec17_30Hz/AQ2am_Dec17_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2pm_Dec17_30Hz/AQ2pm_Dec17_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2am_Dec18_30Hz/AQ2am_Dec18_30Hz.tif'
 b'/media/cat/12TB/in_vivo/tim/yuki/AQ2/tif_files/AQ2pm_Dec18_30Hz/AQ2pm_Dec18_30Hz.tif'
 b'/media/cat/12TB/in_v

In [67]:
plt.plot(data1.mean(0).T, c='blue')
plt.plot(data2.mean(0).T, c= 'red')
plt.show()
         

In [ ]:
# RECOMPUTE DFF ON MASKED DATA

if os.path.exists(fname_04_data_stm)==False:
    # CALL FUNCTION;
    data_stm = compute_DFF_function(
                            root_dir,
                            dff_method, # 'globalAverage' or 'slidingWindow'
                            recording,
                            locs_selected,
                            n_sec_window
                            )
    if data_stm is None:
        print ("data_stm is None", recording)
        return np.zeros((0), 'float32'), np.zeros((0), 'float32') 

    # SAVE data_stm stack
    np.save(fname_04_data_stm, data_stm)

In [7]:
##################################################################
##### COMPUTE ROI TRIAL COURSES - WITH & WITHOUT LOCKOUT #########
##################################################################

names = ['IA1','IA2','IA3','IJ1','IJ2','AR4','AQ2']
names = ['IA2']

from tqdm import tqdm
feature_name = 'code_04'
lockout_window = 10
n_sec_window = 10

for name in names:
    root_dir = '/media/cat/4TBSSD/yuki/'+name
    #recordings = np.loadtxt(root_dir + '/'+name+'.txt',dtype='str')
    
    temp_recs = np.load(root_dir+'/tif_files.npy')
    recordings =[]
    for k in range(len(temp_recs)):
        try:
            recordings.append(str(os.path.split(temp_recs[k])[1][:-4], "utf-8"))
        except:
            recordings.append(os.path.split(temp_recs[k])[1][:-4])
    
    print ("PROCESSING: ", name)
    
    if False:
        import parmap
        res = parmap.map(compute_trial_courses_ROI_code04_trigger, 
                       recordings,
                       root_dir,
                       feature_name,
                       lockout_window,
                       n_sec_window,
                       pm_processes=4,
                       pm_pbar=True)
    else:
        res = []
        for recording in tqdm(recordings):
            print ("recording: ", recording)
            res.append(compute_trial_courses_ROI_code04_trigger(recording,
                                                                root_dir,
                                                                feature_name,
                                                                lockout_window,
                                                                n_sec_window))



  0%|          | 0/44 [00:00<?, ?it/s]

PROCESSING:  IA2
recording:  IA2pm_Feb1_30Hz


  2%|▏         | 1/44 [00:06<04:23,  6.13s/it]

recording:  IA2pm_Feb2_30Hz


  5%|▍         | 2/44 [00:14<04:42,  6.74s/it]

recording:  IA2pm_Feb3_30Hz


  7%|▋         | 3/44 [00:44<09:22, 13.73s/it]

recording:  IA2pm_Feb4_30Hz


  7%|▋         | 3/44 [01:01<13:54, 20.35s/it]


KeyboardInterrupt: 

In [92]:
#########################################################
############## MAKE ORDERED .NZP FILES ##################
#########################################################

# MANUALLY FIND WHICH SESSIONS HAVE LARGE DIFFERENCE OF REWARDED WITH /WITHOUT PULL OVERLAP IN 10SEC WINDOW
# AQ2
selected_sessions_AQ2 = [32,50,53,54,56,57,58,59,63,64,65,66,67,70,71,72,73,74,77,78,79,80,81]
selected_sessions_AQ2 = np.append(selected_sessions_AQ2,np.arange(83,110))
best_sessions_AQ2 = [32,58,91]

# IA1
selected_sessions_IA1 = [2,10,11,13,15,29]
best_sessions_IA1 = [13,15,29]

# IA2
selected_sessions_IA2 = [3,4,13,21,28,29,30,31,32,35,40,43]
best_sessions_IA2 = [3,29,40]

# IA3
selected_sessions_IA3 = [1,2,7,17,21,25,31,32,33,34,35,36,37,38,39,40,41,42,43]
best_sessions_IA3 = [2,34,43]

# IJ1
selected_sessions_IJ1 = [14,18,20,21,22,58]
best_sessions_IJ1 = [14,22,58]

# IJ2
selected_sessions_IJ2 = [2,17,18,19,20,26,33,37,39,40,43]
best_sessions_IJ2 = [20,33,40]

# AR4
selected_sessions_AR4 = [6,12,14,23,26,27,28,30,31,32]
best_sessions_AR4 = [6,23,30]



selected_sessions_animal = selected_sessions_AR4
best_sessions_animal = best_sessions_AR4
    
# CHECK ALL VS. LOCKOUT DATA
names = ['IA1','IA2','IA3','IJ1','IJ2','AR4','AQ2']
names = ['AR4']

save_npz_data(names, selected_sessions_animal, best_sessions_animal)



PROCESSING:  AR4
rec id,      rec name,           all rewarded trials,   10sec lockout rewarded trials (*** good sessions; ####### best 3 sessions
        0       AR4_Oct27pm_15Hz_8x8      (0,)          (0,)
        1       AR4_Oct28pm_15Hz_8x8      (14, 35, 301)          (10, 35, 301)
        2       AR4_Oct29pm_15Hz_8x8      (0,)          (0,)
        3       AR4_Nov3pm_15Hz_8x8      (0,)          (0,)
        4       AR4_Nov3am_15Hz_8x8      (0,)          (0,)
        5       AR4_Nov4am_15Hz_8x8      (92, 35, 301)          (16, 35, 301)
####### 6       AR4_Nov5am_15Hz_8x8      (90, 35, 301)          (38, 35, 301)
        7       AR4_Nov6am_15Hz_8x8      (173, 35, 301)          (23, 35, 301)
        8       AR4_Nov7am_15Hz_8x8      (202, 35, 301)          (21, 35, 301)
        9       AR4_Nov10am_15Hz_8x8      (163, 35, 301)          (11, 35, 301)
        10       AR4_Nov11am_15Hz_8x8      (245, 35, 301)          (19, 35, 301)
        11       AR4_Nov12am_15Hz_8x8      (230, 35, 301)

In [32]:
# # CHECK ALL VS. LOCKOUT DATA
# names = ['IA1','IA2','IA3','IJ1','IJ2','AR4','AQ2']
# names = ['AQ2']

# selected_sessions = [3, 30, 50]

# from tqdm import tqdm
# feature_name = 'code_04'
# lockout_window = 10
# n_sec_window = 10

# for name in names:
#     root_dir = '/media/cat/4TBSSD/yuki/'+name
#     #recordings = np.loadtxt(root_dir + '/'+name+'.txt',dtype='str')
    
#     temp_recs = np.load(root_dir+'/tif_files.npy')
#     recordings =[]
#     for k in range(len(temp_recs)):
#         try:
#             recordings.append(str(os.path.split(temp_recs[k])[1][:-4], "utf-8"))
#         except:
#             recordings.append(os.path.split(temp_recs[k])[1][:-4])
    
#     print ("PROCESSING: ", name)
    
#     print ("rec id,      rec name,           all rewarded trials,   10sec lockout rewarded trials")
#     for ctr,recording in enumerate(recordings):

#         try: 
#             (data_04, data_04_random, data_04_lockout, data_04_lockout_random) = load_trial_courses_ROI_code04_trigger(
#                                                                       recording,
#                                                                       root_dir,
#                                                                       feature_name,
#                                                                       lockout_window,
#                                                                       n_sec_window)
#             if ctr not in selected_sessions:
#                 print ("   ",ctr, "     ", recording,"    ", data_04.shape, "        ", data_04_lockout.shape)
#             else:
#                 print ("***", ctr, "     ", recording,"    ", data_04.shape, "        ", data_04_lockout.shape)
            
#             #print ('')
#         except:
#             pass



PROCESSING:  AQ2
rec id,      rec name,           all rewarded trials,   10sec lockout rewarded trials
*** 3       AQ2am_Dec11_30Hz      (7, 35, 601)          (7, 35, 601)
    4       AQ2pm_Dec14_30Hz      (10, 35, 601)          (9, 35, 601)
    6       AQ2pm_Dec16_30Hz      (27, 35, 601)          (22, 35, 601)
    7       AQ2am_Dec17_30Hz      (21, 35, 601)          (15, 35, 601)
    8       AQ2pm_Dec17_30Hz      (5, 35, 601)          (5, 35, 601)
    9       AQ2am_Dec18_30Hz      (7, 35, 601)          (7, 35, 601)
    10       AQ2pm_Dec18_30Hz      (5, 35, 601)          (5, 35, 601)
    11       AQ2am_Dec21_30Hz      (2, 35, 601)          (2, 35, 601)
    14       AQ2am_Dec28_30Hz      (24, 35, 601)          (13, 35, 601)
    15       AQ2am_Dec29_30Hz      (33, 35, 601)          (27, 35, 601)
    17       AQ2am_Dec31_30Hz      (65, 35, 601)          (51, 35, 601)
    18       AQ2am_Jan4_30Hz      (52, 35, 601)          (42, 35, 601)
    19       AQ2am_Jan5_30Hz      (52, 35, 601)    

In [ ]:
# # MAKE ORDERED .NZP FILES
# # MUST INPUT a temporally ordered.txt file
# fname_list = '/media/cat/4TBSSD/yuki/time_courses/AQ2_ordered.txt'
# fnames = np.loadtxt(fname_list,dtype='str')
# #print (fnames)

# data_list = []
# fnames_data_list = []
# data_random_list = []
# fnames_data_random_list = []
# for fname in fnames:
#     if '04' in fname:
#         data_list.append(np.load(fname))
#         fnames_data_list.append(fname)
#     else:
#         data_random_list.append(np.load(fname))
#         fnames_data_random_list.append(fname)

# np.savez(fname_list[:-4]+'.npz',
#         data_04_lever_pull = data_list,
#         data_04_lever_pull_fnames = fnames_data_list,
#         data_random = data_random_list,
#         data_random_fnames = fnames_data_random_list)


In [ ]:
# GET DISTRIBUTION OF SESSIONS IN EACH ANIMAL

all_files = [
'/media/cat/4TBSSD/yuki/time_courses/AR4_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IA1_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IA2_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IA3_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IJ1_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/IJ2_ordered.npz',
'/media/cat/4TBSSD/yuki/time_courses/AQ2_ordered.npz'
]

tot = 0
for ctr, file_ in enumerate(all_files):
    temp = np.load(file_, allow_pickle=True)
    #fnames = temp['data_04_lever_pull_fnames']
    #print (fnames)
    data = temp['data_04_lever_pull']
    print (file_, " # of sessions: ", len(data))
    tot+=len(data)
    ax=plt.subplot(2,4,ctr+1)
    plt.title(os.path.split(file_)[1])
    
    lengths = []
    for k in range(len(data)):
        lengths.append(len(data[k]))
        #print (data[k].shape)
    y = np.histogram(lengths, np.arange(0,400,10))
    plt.plot(y[1][:-1],y[0])
    plt.ylabel("# of sessions")
    plt.xlabel("# of trials in sessions")
    plt.xlim(0,200)
print ("total: ", tot)

In [ ]:
# SAVE DATA
#np.save(root_dir+"data_random_code_"+str(locs_selected.shape[0])+"trials_.npy", data_stm)

In [66]:
data = np.load('/media/cat/4TBSSD/yuki/IA2/tif_files/IA2pm_Apr6_30Hz/data_random_code_64trials_ROIs.npy')
print (data.shape)

(64, 35, 601)


In [10]:
data= np.load('/media/cat/4TBSSD/yuki/AR4/tif_files.npy')
print (data)

['/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Oct27pm_15Hz_8x8/AR4_Oct27pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Oct28pm_15Hz_8x8/AR4_Oct28pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Oct29pm_15Hz_8x8/AR4_Oct29pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov3pm_15Hz_8x8/AR4_Nov3pm_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov3am_15Hz_8x8/AR4_Nov3am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov4am_15Hz_8x8/AR4_Nov4am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov5am_15Hz_8x8/AR4_Nov5am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov6am_15Hz_8x8/AR4_Nov6am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov7am_15Hz_8x8/AR4_Nov7am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4_Nov10am_15Hz_8x8/AR4_Nov10am_15Hz_8x8.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/AR4/tif_files/AR4

In [30]:
print (np.load('/media/cat/4TBSSD/yuki/IA1/tif_files.npy'))

['/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/IA1pm_Feb1_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb2_30Hz/IA1pm_Feb2_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb3_30Hz/IA1pm_Feb3_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb4_30Hz/IA1pm_Feb4_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb5_30Hz/IA1pm_Feb5_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb9_30Hz/IA1pm_Feb9_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb10_30Hz/IA1pm_Feb10_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb11_30Hz/IA1pm_Feb11_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb12_30Hz/IA1pm_Feb12_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb15_30Hz/IA1pm_Feb15_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_files/IA1pm_Feb16_30Hz/IA1pm_Feb16_30Hz.tif'
 '/media/cat/12TB/in_vivo/tim/yuki/IA1/tif_f

In [17]:
import h5py
filename = "/home/cat/Downloads/AR4_2014-12-02_12-56-34.975DLC_resnet50_yuki_leverJul21shuffle1_200000.h5"

f = h5py.File(filename, "r")
# List all groups
print("Keys: %s" % f.keys())
a_group_key = list(f.keys())[0]

# Get the data
data = list(f[a_group_key])

data2 = f['df_with_missing']['table']

Keys: <KeysViewHDF5 ['df_with_missing']>


In [31]:
print (data2[0][1].shape)

traces = []
for k in range(len(data2)):
    traces.append(data2[k][1].reshape(-1,3))
    

(21,)


In [33]:
traces = np.array(traces)
print (traces.shape)

(20066, 7, 3)


In [11]:
files = np.load('/media/cat/4TBSSD/yuki/IJ1/tif_files.npy')
print (files[0].decode("utf-8"))

AttributeError: 'str' object has no attribute 'decode'